In [1]:
#pip install NRCLex

In [2]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
from preprocessing import processing
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# Import earning calls
import glob
list_articles = glob.glob("data/earning_call/*")
texts = []
earning_call_id = []
for s in list_articles:
    earning_call_id.append(s[-7:])
    with open(s) as f:
        texts.append(f.read())

In [4]:
# Get earning call id from file name
mypath= "data/earning_call"
from os import listdir
from os.path import isfile, join
earning_call_id = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
# Get dates
data_path = "data/date"
with open(data_path,"rb") as file :
    date = pickle.load(file)
print(len(date))
print(len(earning_call_id))

1200
1415


In [6]:
# Get tickers
data_path = "data/id_to_ticker"
with open(data_path,"rb") as file :
    id_mapping = pickle.load(file)

In [15]:
# Test : create dataframe with earning calls informations
df1 = pd.DataFrame(data={'id': earning_call_id})
df1['date']=earning_call_id
df1=df1.replace({'date': date})
df1['ticker']= earning_call_id
df1=df1.replace({'ticker': id_mapping})
df1= df1.sort_values(by=['id'], ascending=False)
display(df1[:1000])

,id,date,ticker
751,4512747,2022-05-18,AYX
76,4512744,2022-05-18,TEF
1328,4512738,2022-05-18,DTRUY
7,4512728,2022-05-17,ENGIY
791,4512725,2022-05-17,DOCS
...,...,...,...
513,4510142,2022-05-11,EVAX
1168,4510141,2022-05-11,PANL
1115,4510140,2022-05-11,AGFY
371,4510128,2022-05-11,XERS


In [8]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

tickers = df1['ticker'].tolist()
start = datetime.datetime(2022,5,16)
end = datetime.datetime(2022,5,20)

# create empty dataframe
stock_final = pd.DataFrame()
# iterate over each symbol
for i in tickers[:5]:  
    
    # print the symbol which is being downloaded
    print( str(tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None


0 : AYX,1 : TEF,2 : DTRUY,3 : ENGIY,4 : DOCS,

In [9]:
stock_final

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2022-05-16,58.520000,59.865002,55.709999,55.799999,55.799999,703300,AYX
2022-05-17,57.000000,58.240002,54.119999,56.200001,56.200001,587900,AYX
2022-05-18,55.400002,56.230000,52.790001,53.570000,53.570000,678400,AYX
2022-05-16,4.890000,4.950000,4.860000,4.950000,4.950000,2578500,TEF
2022-05-17,4.980000,5.050000,4.960000,5.050000,5.050000,1598100,TEF
2022-05-18,4.960000,4.970000,4.890000,4.920000,4.920000,1307200,TEF
2022-05-16,14.080000,14.280000,13.980000,14.241000,14.241000,82600,DTRUY
2022-05-17,15.190000,15.500000,15.010000,15.276000,15.276000,60300,DTRUY
2022-05-18,15.750000,15.750000,15.110000,15.180000,15.180000,16100,DTRUY


In [10]:
# Preprocessing
n= 10
texts = [processing(x) for x in texts][:n]

In [11]:
#N=len(texts)

for i in range(n):
    # creating objects
    text=texts[i]
    emo = LeXmo.LeXmo(text)
    emo.pop('text', None)
    emo_array = np.array(list(emo.items()))
    score = emo_array[:,1]
    if i==0:
        con = score
    else :
        con = np.vstack((con, score))

In [12]:
# Create dataframe
score = np.reshape(con, (n, 10))
column=['anger','anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']
df = pd.DataFrame(score, columns = column)

In [16]:
number_words = np.zeros(n)
for i in range(n):
    number_words[i] = len(texts[i])
df['number_words']=number_words
df['id'] = earning_call_id[:n]
df['date']=earning_call_id[:n]
df=df.replace({'date': date})
df['ticker']= earning_call_id[:n]
df=df.replace({'ticker': id_mapping})
df= df.sort_values(by=['id'], ascending=False)
display(df)

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,number_words,id,date,ticker
7,0.008483853311439518,0.0481663929939792,0.0021893814997263274,0.010673234811165846,0.02435686918445539,0.029556650246305417,0.09688013136288999,0.014230979748221127,0.00875752599890531,0.05172413793103448,28106.0,4512728,2022-05-17,ENGIY
2,0.008119079837618403,0.05087956698240866,0.0040595399188092015,0.011096075778078484,0.020838971583220567,0.022192151556156968,0.09769959404600811,0.009201623815967524,0.009742895805142085,0.04817320703653586,27264.0,4512525,2022-05-17,SONVF
5,0.011382113821138212,0.04390243902439024,0.0040650406504065045,0.00975609756097561,0.01788617886178862,0.023577235772357725,0.10650406504065041,0.01056910569105691,0.012195121951219513,0.05853658536585366,9243.0,4510888,2022-05-12,HCDI
0,0.0046248715313463515,0.046762589928057555,0.0005138746145940391,0.0046248715313463515,0.018499486125385406,0.013874614594039054,0.0750256937307297,0.0051387461459403904,0.013874614594039054,0.03597122302158273,14598.0,4510424,2022-05-11,LNNNF
3,0.004914933837429111,0.052930056710775046,0.0011342155009451795,0.007183364839319471,0.022684310018903593,0.019281663516068054,0.09640831758034027,0.011720226843100189,0.010964083175803403,0.05330812854442344,19476.0,4510272,2022-05-11,VIV
8,0.011375947995666305,0.048212351029252434,0.005417118093174431,0.009209100758396533,0.024918743228602384,0.029794149512459372,0.1056338028169014,0.013542795232936078,0.010292524377031419,0.06717226435536294,14245.0,4509536,4509536,4509536
1,0.01146384479717813,0.0582010582010582,0.007495590828924162,0.007054673721340388,0.040123456790123455,0.029541446208112873,0.1027336860670194,0.009259259259259259,0.021164021164021163,0.06128747795414462,16564.0,4509509,4509509,4509509
9,0.009034589571502324,0.05085183273102736,0.002065049044914817,0.006969540526587507,0.02194114610221993,0.01961796592669076,0.08466701084150749,0.005678884873515746,0.014197212183789365,0.059886422302529684,28539.0,4509394,4509394,4509394
6,0.009439200444197668,0.06274292059966685,0.000555247084952804,0.011660188784008884,0.03387007218212104,0.02387562465297057,0.1254858411993337,0.012770682953914493,0.014436424208772903,0.08217656857301499,13526.0,4509360,4509360,4509360
4,0.0055514433752775726,0.05773501110288675,0.0025906735751295338,0.011102886750555145,0.021095484826054774,0.015173945225758698,0.10510732790525537,0.007031828275351592,0.014433752775721688,0.058105107327905255,20509.0,4509199,4509199,4509199


In [14]:
print(date.get('4509199'))

None


# A faire pour cette méthode : 
- classer les earning calls par quarter et par stock index
- importer les stock returns et garder seulement ceux des earning calls (mapping permno/ stock index)

# liens utiles

https://betterprogramming.pub/unlocking-emotions-in-text-using-python-6d062b48d71f